In [34]:
import deribit_data as dm
from option import Option
import numpy as np
from utils import *
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go   

In [35]:
data = dm.Scraper(currency='BTC')

In [36]:
raw_df = data.collect_data(max_workers = 15, save_csv = False)

Data Collected


In [37]:
raw = raw_df.copy()

In [38]:
option = Option(raw, 'C')

/Users/pierrebelamri/Documents/GitHub/deribit_scraping/utils.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mid_iv'] = (df.bid_iv + df.ask_iv)/2
/Users/pierrebelamri/Documents/GitHub/deribit_scraping/utils.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_T'] = df['_T'].apply(to_ts) - datetime.datetime.timestamp(datetime.datetime.today())#btc_subset['timestamp'].apply(ms_to_s)
/Users/pierrebelamri/Documents/GitHub/deribit_scraping/utils.py:129: SettingWithCopyWarning: 
A value is trying 

In [39]:
#option.init_merton(m = 0.625038301534241, lam =0.0010591027301815744, v = 0.511097066109511,  )
option.init_merton(reset=True)

In [40]:
#mkt_data=option.df[["mid","K","_T"]]
#mkt_data["mid"]=mkt_data["mid"]*option.df["S"]
#option.df["S"].[0]

In [41]:
import numpy as np
import math


class option_2():
    def __init__(self, theta, data):
        self.theta = theta
        self.S_0 = data[0]
        self.r = data[1]
        self.M = int(data[2])
        self.deg = int(data[3])
        self.K = data[4]
        self.T = data[5]
        self.N = int(data[6])
        self.it = int(data[7])
    def path(self):
        dt = self.T/self.N
        paths = np.random.multivariate_normal([0, 0], [[1, self.theta[2]], [self.theta[2], 1]], self.N + 1).transpose()
        paths[0, 0] = self.S_0
        paths[1, 0] = self.theta[0]
        for j in range(0, self.N):
            paths[1, j + 1] = paths[1, j] + self.theta[3]*(self.theta[1] - max(paths[1, j], 0))*dt \
                               + self.theta[4]*np.sqrt(max(0, paths[1, j])*dt)*paths[1, j + 1]
            paths[0, j + 1] = paths[0, j]*np.exp((self.r - max(paths[1, j], 0)/2)*dt \
                                                 + np.sqrt(max(paths[1, j], 0)*dt)*paths[0, j + 1])
        return paths
    def step(self, price, vol):
        dt = self.T / self.N
        norm_var = np.random.multivariate_normal([0, 0], [[1, self.theta[2]], [self.theta[2], 1]], self.it).transpose()
        vol[vol < 0] = 0
        sqrt_vol = np.sqrt(vol)
        next_vol = vol + self.theta[3] * (self.theta[1] - vol) * dt \
                   + self.theta[4] * sqrt_vol * np.sqrt(dt) * norm_var[0, :]
        exp_vol = (self.r - vol / 2) * dt + sqrt_vol * np.sqrt(dt) * norm_var[1, :]
        next_price = price * np.exp(exp_vol)
        return np.concatenate([[next_vol], [next_price]])
    def eur_option(self):
        return hf.eur_call_heston_price(self.theta, self.K, self.T, self.S_0, self.r, self.M, self.deg)
    def asian_option(self):
        price = np.empty(self.it)
        price.fill(self.S_0)
        vol = np.empty(self.it)
        vol.fill(self.theta[0])
        sum = np.zeros(self.it)
        for j in range(0, self.N):
            next = self.step(price, vol)
            sum = sum + next[1, :]
            vol = next[0, :]
            price = next[1, :]
        asian_price = sum/self.N - self.K
        asian_price[asian_price < 0] = 0
        return asian_price.sum()*np.exp(- self.r*self.T)/self.it


In [71]:
import numpy as np
import math as math
import cmath
#from uploading_data import mkt_data, S_0, r, M, deg
from scipy import optimize
theta_0 = np.array([0.08, 0.10, 0.8, 3, 0.25])
S_0 =34400
r = 0.01
M = 100
deg = 64
#mkt_data=option.df[["mid", "K","_T"]]
#mkt_data["mid"]=mkt_data["mid"]*option.df["S"]
# SECTION 1 - Functions, which are necessary to compute price of the european option for given parameters.
def ksi(theta, u):
    return theta[3] - theta[4]*theta[2]*u*1j
def d(theta, u):
    return cmath.sqrt(ksi(theta, u)*ksi(theta, u) + math.pow(theta[4], 2)*(u*u + u*1j))
def g1(theta, u):
    return (ksi(theta, u) + d(theta, u))/(ksi(theta, u) - d(theta, u))
def A1(theta, u, t):
    return (u*u + 1j*u)*cmath.sinh(d(theta, u)*t/2)
def A2(theta, u, t):
    return d(theta, u)*cmath.cosh(d(theta, u)*t/2)/theta[0] + ksi(theta, u)*cmath.sinh(d(theta, u)*t/2)/theta[0]
def A(theta, u, t):
    return A1(theta, u, t)/A2(theta, u, t)
def B(theta, u, t):
    return d(theta, u)*cmath.exp(theta[3]*t/2)/(A2(theta, u, t)*theta[0])
def D(theta, u, t):
    return cmath.log(d(theta, u)/theta[0]) + theta[3]*t/2 - cmath.log(A2(theta, u, t))
# Equation (18) p. 9 - characteristic function, which we are going to use in out project
def char_function(theta, u, t, S_0, r):
    return cmath.exp(1j*u*(np.log(S_0*np.exp(r*t)/S_0)) - t*theta[3]*theta[1]*theta[2]*1j*u/theta[4] - A(theta, u, t) + \
                     2*theta[3]*theta[1]*D(theta, u, t)/math.pow(theta[4], 2))
# integrate_char_function - integrals computed by means of Gauss-Legendre Quadrature
def integrate_char_function(theta, K, t, S_0, r, i, M, deg):
    x, w = np.polynomial.legendre.leggauss(deg)
    u = (x[0]+1)*0.5*M
    value = w[0]*cmath.exp(-1j*u*np.log(K/S_0))/(1j*u)*char_function(theta, u - i, t, S_0, r)
    for j in range(1, deg): # deg - number of nodes
        u = (x[j] + 1)*0.5*M
        value = value + w[j]*cmath.exp(-1j*u*np.log(K/S_0))/(1j*u)*char_function(theta, u - i, t, S_0, r)
    value = value*0.5*M
    return value.real
# eur_call_heston_price - Equation (9)
def eur_call_heston_price(K, t, S_0, r=0.0, M=100, deg=64,theta=np.array([0.41055433, 2.87649559, 1.0035074 , 1.14520439, 2.15878211])):
    return (S_0 - np.exp(-r*t)*K)/2 + np.exp(-r*t)/np.pi*(S_0*integrate_char_function(theta, K, t, S_0, r, 1j, M, deg) - \
                                                                K*integrate_char_function(theta, K, t, S_0, r, 0, M, deg))
# SECTION 2 - functions, which are necessary to compute gradient of characteristic function
def h_1(theta, u, t):
    return -A(theta, u, t)/theta[0]
def h_2(theta, u ,t):
    return 2*theta[3]*D(theta, u, t)/math.pow(theta[4], 2) - t*theta[3]*theta[2]*1j*u/theta[4]
def h_3(theta, u, t):
    return - A_rho(theta, u, t) + 2*theta[3]*theta[1]*(d_rho(theta,u) - d(theta, u)*A2_rho(theta, u, t)/A2(theta,u,t))/ \
                                  (theta[4]*theta[4]*d(theta, u)) - t*theta[3]*theta[1]*1j*u/theta[4]
def h_4(theta, u, t):
    return A_rho(theta, u, t)/(theta[4]*1j*u) + 2*theta[1]*D(theta, u, t)/(theta[4]*theta[4]) + \
           2*theta[3]*theta[1]*B_kappa(theta, u, t)/(theta[4]*theta[4]*B(theta, u, t)) - \
           t*theta[1]*theta[2]*1j*u/theta[4]
def h_5(theta, u, t):
    return - A_sigma(theta, u, t) - 4*theta[3]*theta[1]*D(theta, u, t)/(math.pow(theta[4], 3)) + \
           2*theta[3]*theta[1]*(d_rho(theta, u) - d(theta, u)*A2_sigma(theta, u, t)/A2(theta, u, t))/ \
           (theta[4]*theta[4]*d(theta, u)) + t*theta[3]*theta[1]*theta[2]*1j*u/(theta[4]*theta[4])
def d_rho(theta, u):
    return - ksi(theta, u)*theta[4]*1j*u/d(theta, u)
def A2_rho(theta, u, t):
    return - theta[4]*1j*u*(2 + t*ksi(theta, u))*(ksi(theta, u)*cmath.cosh(d(theta, u)*t/2) + \
                                                  d(theta, u)*cmath.sinh(d(theta, u)*t/2))/(2*d(theta, u)*theta[0])
def B_rho(theta, u, t):
    return cmath.exp(theta[3]*t/2)*(d_rho(theta, u)/A2(theta, u, t) - \
                                    A2_rho(theta, u, t)/(A2(theta,u,t)*A2(theta,u,t)))/theta[0]
def A1_rho(theta, u, t):
    return - 1j*u*(u*u + 1j*u)*t*ksi(theta, u)*theta[4]*cmath.cosh(d(theta, u)*t/2)/(2*d(theta, u))
def A_rho(theta, u, t):
    return A1_rho(theta, u, t)/A2(theta, u, t) - A2_rho(theta, u, t)*A(theta, u, t)/A2(theta, u, t)
def A_kappa(theta, u, t):
    return 1j*A_rho(theta, u, t)/(u*theta[4])
def B_kappa(theta, u, t):
    return B_rho(theta, u, t)*1j/(theta[4]*u) + t*B(theta, u, t)/2
def d_sigma(theta, u):
    return (theta[2]/theta[4] - 1/ksi(theta, u))*d_rho(theta, u) + theta[4]*u*u/d(theta, u)
def A1_sigma(theta, u, t):
    return (u*u + 1j*u)*t*d_sigma(theta, u)*cmath.cosh(d(theta, u)*t/2)/2
def A2_sigma(theta, u, t):
    return theta[2]*A2_rho(theta, u, t)/theta[4] - (2 + t*ksi(theta, u))*A1_rho(theta, u, t)/ \
                                                   (1j*u*t*ksi(theta, u)*theta[0]) + theta[4]*t*A1(theta, u, t)/(2*theta[0])
def A_sigma(theta, u, t):
    return A1_sigma(theta, u, t)/A2(theta, u, t) - A(theta, u, t)*A2_sigma(theta, u, t)/A2(theta, u, t)
def h(theta, u, t, which):
    if which == 1:
        return h_1(theta, u, t)
    if which == 2:
        return h_2(theta, u, t)
    if which == 3:
        return h_3(theta, u, t)
    if which == 4:
        return h_4(theta, u, t)
    if which == 5:
        return h_5(theta, u, t)
# integrate_grad_function - integrals computed by means of Gauss-Legendre Quadrature
def integrate_grad_function(theta, K, t, S_0, r, i, M, deg, which):
    x, w = np.polynomial.legendre.leggauss(deg)
    u = (x[0] + 1)*0.5*M
    value = w[0]*cmath.exp(-1j*u*np.log(K/S_0))/(1j*u)*char_function(theta, u - i, t, S_0, r)*h(theta, u - i, t, which)
    for j in range(1, deg):
            u = (x[j] + 1)*0.5*M
            value = value + w[j]*cmath.exp(-1j*u*np.log(K/S_0))/(1j*u)*char_function(theta, u - i, t, S_0, r)* \
                            h(theta, u - i, t, which)
    return value.real
# grad_heston_price - Equation (22)
def grad_heston_price(theta, t, K, S_0, r, M, deg):
    first_int = np.array(integrate_grad_function(theta, K, t, S_0, r, 1j, M, deg, 1))
    second_int = np.array(integrate_grad_function(theta, K, t, S_0, r, 0, M, deg, 1))
    for i in range(2, 6):
        first_int = np.append(first_int, integrate_grad_function(theta, K, t, S_0, r, 1j, M, deg, i))
        second_int = np.append(second_int, integrate_grad_function(theta, K, t, S_0, r, 0, M, deg, i))
    return (np.exp(-r*t)/np.pi)*(first_int - K*second_int)
def r_function(theta):
    r_vector = np.array(eur_call_heston_price(theta, mkt_data[0, 1], mkt_data[0, 2], S_0, r, M, deg) - \
                        mkt_data[0, 0])
    for i in range(1, len(mkt_data)):
        r_vector = np.append(r_vector, eur_call_heston_price(theta, mkt_data[i, 1], mkt_data[i, 2], S_0, r, M, deg) - \
                             mkt_data[i, 0])
    return r_vector
def calibration_heston(theta):
    return  optimize.least_squares(r_function, theta, method='lm').x

In [43]:
#theta

theta_calibré = array([0.41055433, 2.87649559, 1.0035074 , 1.14520439, 2.15878211])

In [44]:
#theta = calibration_heston(theta_0)
theta = np.array([0.41055433, 2.87649559, 1.0035074 , 1.14520439, 2.15878211])

In [45]:
#mkt_data.to_csv (r'mkt_data.csv', index = False, header=True)

In [46]:
option.append_imp_vol_to_df()
option.append_loc_vol_to_df()

/Users/pierrebelamri/Documents/GitHub/deribit_scraping/option.py:210: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(num/denom)
/Users/pierrebelamri/Documents/GitHub/deribit_scraping/option.py:210: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(num/denom)


In [47]:
option.append_BS_price()

In [82]:
option.df.dropna(inplace=True)
option.df = option.df.drop(option.df[option.df.BS_PRICE >= .35].index) #a revoir

#option.df = option.df.drop(option.df[option.df.IV_moneyness >= .5*10**6].index)

In [49]:
#option.optimize_merton(tol = 1e-6, max_iter = 600)

In [50]:
option.append_Merton_price()

In [51]:
#option.append_Merton_priceMC()

In [52]:
option.df.head()

,S,K,V,_T,bids,asks,last_price,mark_price,option_type,mid_iv,mark_iv,mid,moneyness,I_VOL_MERTON,I_VOL_BS,IV_moneyness,L_VOL,BS_PRICE,MERTON_PRICE
377,34387.55,35000.0,108.4,0.001019,0.010272,0.017789,0.0115,0.011468,C,102.075,101.90,0.014031,0.982501,1.605525e-01,0.161456,212983.789584,0.965960,0.014139,0.014132
135,34411.38,36000.0,281.8,0.001019,0.003247,0.008403,0.0045,0.004523,C,102.540,102.75,0.005825,0.955872,3.636918e-01,0.367806,93558.443133,1.404965,0.005847,0.005843
215,34409.57,37000.0,47.9,0.001019,0.000662,0.003117,0.0005,0.001448,C,103.855,103.77,0.001890,0.929988,5.237420e-01,0.542971,63372.773787,1.659231,0.001895,0.001892
173,34412.55,38000.0,26.9,0.001019,0.000000,0.001130,0.0005,0.000389,C,61.380,104.53,0.000565,0.905593,4.768374e-07,0.691008,49800.544295,1.827146,0.000566,0.000954
371,34387.27,36000.0,73.8,0.003759,0.009404,0.011888,0.0090,0.010993,C,99.820,99.80,0.010646,0.955202,1.960113e-01,0.201321,170808.199175,0.758731,0.010781,0.010737


In [72]:
#eur_call_heston_price(theta, K, t, S_0, r, M, deg)
#eur_call_heston_price(theta, 36000.0, 0.001019, 34411.38, 0.01, 100, 64)

heston_prices = np.vectorize(eur_call_heston_price)(K = option.df['K'],
                                                    t = option.df['_T'],
                                                    S_0=option.df['S'],
                                                    )

In [87]:

option.df = option.df.drop(option.df[option.df.heston_prices < 0].index)

# ==================

In [ ]:
option.df['MERTON_PRICE MC']=option.df['MERTON_PRICE MC']*option.df.S[1]

In [88]:
x = option.df['K']
y = option.df['_T']
z = option.df['BS_PRICE']
z2 = option.df['mid']
z3 = option.df['MERTON_PRICE']
z4 = option.df['heston_prices']/option.df['K']


#fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='BS price')])

fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='Black&Scholes model Price'),
                      go.Scatter3d(x=x, y=y, z=z2, mode='markers', name='Real Market Price'),
                      go.Scatter3d(x=x, y=y, z=z3, mode='markers', name='Merton Model Price'),
                      go.Scatter3d(x=x, y=y, z=z4, mode='markers', name='Heston Model Price')])


fig.update_scenes(xaxis_title_text='Strike', yaxis_title_text='Exp', zaxis_title_text='Price') 
fig.show()

rms_merton = mean_squared_error(option.df['mid'], option.df['MERTON_PRICE'], squared=False)
rms_bs = mean_squared_error(option.df['mid'], option.df['BS_PRICE'], squared=False)
rms_hest = mean_squared_error(z2, z4, squared=False)



print('Root Mean Squared Error | MERTON = {}%'.format(round(rms_merton*100,6)))
print('Root Mean Squared Error | B&S    = {}%'.format(round(rms_bs*100,6)))
print('Root Mean Squared Error | HESTON    = {}%'.format(round(rms_hest*100,6)))

Root Mean Squared Error | MERTON = 1.717893%
Root Mean Squared Error | B&S    = 2.67435%
Root Mean Squared Error | HESTON    = 1.958572%
